# Analysis of Axial Scan for PSI Paper
This is the most up to date (as of 2020-06-19) analysis of the axial scan data, with temperature minimisation implemented, to make figures to put into the PSI paper

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import xarray as xr 
import scipy.stats as stat
from scipy.interpolate import interp1d, UnivariateSpline
import collections
import sys
import os
import glob
import re
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.magnum.database as ut
import flopter.magnum.utils as mgut
import flopter.core.fitters as fts
import flopter.core.constants as c

In [2]:
# Create analysed dataset metadata 

path_to_datasets = '/home/jleland/data/external/magnum/'
# path_to_datasets = '/home/jleland/data/externy/magnum/'
# path_to_analysed_datasets = 'analysed_2'
# path_to_analysed_datasets = 'analysed_3'
# path_to_analysed_datasets = 'phobos_test'
# path_to_analysed_datasets = 'test'
# path_to_analysed_datasets = 'analysed_3_downsampled'
# path_to_analysed_datasets = 'analysed_4'
path_to_analysed_datasets = 'analysed_4_downsampled'
os.chdir(path_to_datasets)

In [3]:
magnum_probes = lp.MagnumProbes()

## Load adc file metadata

In [4]:
os.chdir('/home/jleland/data/external/magnum/')
# os.chdir('/home/jleland/data/externy/magnum/')
meta_data_ds = xr.open_dataset('all_meta_data.nc')
print(meta_data_ds)

<xarray.Dataset>
Dimensions:                 (shot_number: 523, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) int32 0 1 2 3 4 ... 519 520 521 522
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Data variables:
    adc_filename            (shot_number) object ...
    ts_density              (shot_number, ts_radial_pos) float64 ...
    ts_temperature          (shot_number, ts_radial_pos) float64 ...
    ts_d_density            (shot_number, ts_radial_pos) float64 ...
    ts_d_temperature        (shot_number, ts_radial_pos) float64 ...
    adc_folder              (shot_number) object ...
    adc_calibration_index   (shot_number) object ...
    adc_4_

## Start of Analysis

In [5]:
super_title = 'H axial scan @ 0.8T, tilt=10'
axial_scan_indices = [132,148,149,150,151,152,153,154,155,156,157,158,159,160,161,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189]

The below is a set of all non-numerical variables which, for some reason, now have to be dropped before we can run where on a reduced dataset. 

In [6]:
droppable_vars = {
    'adc_4_probe',
    'adc_5_probe',
    'adc_calibration_index',
    'adc_end_time',
    'adc_filename',
    'adc_folder',
    'adc_index',
    'adc_time',
    'shot_end_time',
    'sweep_time',
    'ts_number',
    'ts_time',
    'ts_timestamp'
}

In [7]:
axial_scan_ds = mgut.get_dataset_from_indices(axial_scan_indices, anglescan_fl=False, 
                                              path_to_analysed_datasets=path_to_analysed_datasets, 
                                              average_direction_fl=False).drop(droppable_vars)

In [8]:
probes = np.array(axial_scan_ds.probe.values, ndmin=1)

if len(probes) == 1:
    fig, ax = plt.subplots()
    axial_scan_ds.set_coords('voltage')['current'].plot.line(x='voltage', hue='shot_number', ax=ax)
else:
    fig, ax = plt.subplots(len(probes))
    for i, probe in enumerate(probes):
        ds = axial_scan_ds.sel(probe=probe)
        ds.set_coords('voltage')['current'].plot.line(x='voltage', hue='shot_number', ax=ax[i])

In [9]:
fit_data_df = mgut.fit_magnum_ds(axial_scan_ds, probes=('L', 'S'), threshold=None, scan_param='shot_number', plot_fl=False, mode=3, trimming_vals=(0.3, 0.0, 0.02))

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:215: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:215: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.

In [10]:
# ds = axial_scan_ds.sel(shot_number=148, probe='L')
# ds = ds.where(np.isfinite(ds), drop=True)
# ds.tilt.values

In [10]:
fit_data_df

,shot_number,probe,B,ts_temp,ts_dens,fit_success_fl,temp,d_temp,isat,d_isat,...,d_a,v_f,d_v_f,dens,d_dens,chi2,reduced_chi2,phi,temp_phi,temp_fit_phi
0,127,L,0.8,3.0018971964737067,1.5935223487373072e+19,True,2.493055,0.065139,0.005947,0.000067,...,0.002041,-20.926419,0.051182,3.995699e+17,2.227157e+16,154.945513,4.695319,-16.186991,1.499641,1.414923
1,127,S,0.8,3.0018971964737067,1.5935223487373072e+19,True,3.182550,0.038946,0.071494,0.000248,...,0.000489,-24.463086,0.022999,6.528068e+18,4.378195e+17,126.526836,4.081511,-19.626622,1.490810,1.443893
2,143,L,0.8,nan,nan,True,2.386749,0.121211,0.004320,0.000072,...,0.003503,-17.653438,0.071258,2.966050e+17,1.812243e+16,152.196349,4.476363,-9.827761,2.403972,2.336301
3,143,S,0.8,nan,nan,True,3.151693,0.040212,0.067642,0.000187,...,0.000382,-25.823344,0.028841,6.206519e+18,4.162025e+17,52.768704,1.758957,-21.496799,1.326955,1.291660
4,144,L,0.8,nan,nan,True,2.343736,0.118423,0.004508,0.000069,...,0.003209,-17.382268,0.065157,3.123980e+17,1.894967e+16,144.957988,4.263470,-11.871407,1.716508,1.645229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,182,S,0.8,nan,nan,True,3.475851,0.034462,0.057227,0.000164,...,0.000438,-26.720636,0.020064,4.999991e+18,3.347127e+17,69.624275,2.320809,-21.608728,1.573303,1.526125
80,183,L,0.8,nan,nan,True,2.078496,0.063153,0.009468,0.000094,...,0.001078,-9.248131,0.043159,6.966556e+17,3.903085e+16,71.630385,1.885010,-3.700838,1.708104,1.656106
81,183,S,0.8,nan,nan,True,3.380313,0.047637,0.056278,0.000130,...,0.000363,-26.895619,0.041896,4.986142e+18,3.346129e+17,39.093764,1.348061,-21.527325,1.603068,1.602667
82,184,L,0.8,nan,nan,True,2.040450,0.084109,0.005360,0.000054,...,0.002300,-21.057657,0.043027,3.980264e+17,2.298480e+16,91.349095,2.854659,-16.186092,1.517476,1.454372


In [11]:
ds = fit_data_df.to_xarray()

ds = ds.set_coords(['probe', 'shot_number']).swap_dims({'index':'shot_number'}).drop('index')
s_shots = ds.where(ds.probe=='S', drop=True).drop('probe')
l_shots = ds.where(ds.probe=='L', drop=True).drop('probe')

probes_da = xr.DataArray(['S', 'L'], dims=['probe'], name='probe')

ds_stacked = xr.concat([s_shots, l_shots], dim=probes_da)
ds_stacked

<xarray.Dataset>
Dimensions:         (probe: 2, shot_number: 42)
Coordinates:
  * shot_number     (shot_number) object 127 143 144 145 146 ... 181 182 183 184
  * probe           (probe) <U1 'S' 'L'
Data variables:
    B               (probe, shot_number) float64 0.8 0.8 0.8 0.8 ... 0.8 0.8 0.8
    ts_temp         (probe, shot_number) object 3.0018971964737067 nan ... nan
    ts_dens         (probe, shot_number) object 1.5935223487373072e+19 ... nan
    fit_success_fl  (probe, shot_number) object True True True ... True True
    temp            (probe, shot_number) float64 3.183 3.152 ... 2.078 2.04
    d_temp          (probe, shot_number) float64 0.03895 0.04021 ... 0.08411
    isat            (probe, shot_number) float64 0.07149 0.06764 ... 0.00536
    d_isat          (probe, shot_number) float64 0.0002479 ... 5.369e-05
    a               (probe, shot_number) float64 0.02151 0.02332 ... 0.08151
    d_a             (probe, shot_number) float64 0.0004888 0.0003818 ... 0.0023
    v_f             (probe, shot_number) float64 -24.46 -25.82 ... -9.248 -21.06
    d_v_f           (probe, shot_number) float64 0.023 0.02884 ... 0.04303
    dens            (probe, shot_number) float64 6.528e+18 ... 3.98e+17
    d_dens          (probe, shot_number) float64 4.378e+17 ... 2.298e+16
    chi2            (probe, shot_number) float64 126.5 52.77 ... 71.63 91.35
    reduced_chi2    (probe, shot_number) float64 4.082 1.759 ... 1.885 2.855
    phi             (probe, shot_number) float64 -19.63 -21.5 ... -3.701 -16.19
    temp_phi        (probe, shot_number) float64 1.491 1.327 ... 1.708 1.517
    temp_fit_phi    (probe, shot_number) float64 1.444 1.292 ... 1.656 1.454

In [12]:
ds_combined = xr.merge([axial_scan_ds, ds_stacked])
ds_combined

<xarray.Dataset>
Dimensions:                 (probe: 2, shot_number: 42, time: 50, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'S' 'L'
  * shot_number             (shot_number) int32 127 143 144 145 ... 182 183 184
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    (shot_number) float64 10.0 10.0 10.0 ... 10.0 10.0
Dimensions without coordinates: time
Data variables:
    voltage                 (shot_number, probe, time) float64 6.129 ... -94.59
    current                 (shot_number, probe, time) float64 0.3152 ... -0.01195
    shot_time               (shot_number, probe, time) float64 5.324 ... 4.329
    start_time              (shot_number, probe) float64 5.322 5.322 ... 4.327
    d_current               (shot_number, probe, time) float64 0.001015 ... 7.123e-05
    std_current             (shot_number, probe, time) float64 0.04219 ... 0.002598
    ts_density              (shot_number, ts_radial_pos) float64 ...
    ts_temperature          (shot_number, ts_radial_pos) float64 ...
    ts_d_density            (shot_number, ts_radial_pos) float64 ...
    ts_d_temperature        (shot_number, ts_radial_pos) float64 ...
    ts_temp_max             (shot_number) float64 ...
    ts_dens_max             (shot_number) float64 ...
    ts_fl                   (shot_number) bool ...
    shot_source_current     (shot_number) float64 ...
    shot_b_field            (shot_number) float64 ...
    shot_tilt               (shot_number) float64 9.991 9.987 ... 9.987 9.984
    shot_target_pos         (shot_number) float64 ...
    shot_target_rot         (shot_number) float64 ...
    shot_target_pumpspeed   (shot_number) int32 ...
    shot_source_pumpspeed   (shot_number) int32 ...
    shot_heating_pumpspeed  (shot_number) int32 ...
    shot_source_pos         (shot_number) float64 ...
    shot_deuterium_gf       (shot_number) float64 ...
    shot_hydrogen_gf        (shot_number) float64 ...
    shot_helium_gf          (shot_number) float64 ...
    shot_hydrogen_puffed    (shot_number) float64 ...
    shot_helium_puffed      (shot_number) float64 ...
    adc_timestamp           (shot_number) float64 ...
    adc_4_coax              (shot_number) float64 ...
    adc_4_shunt_resistance  (shot_number) float64 ...
    adc_5_coax              (shot_number) float64 ...
    adc_5_shunt_resistance  (shot_number) float64 ...
    adc_calibration_fl      (shot_number) float64 ...
    shot_fl                 (shot_number) float64 ...
    acquisition_length      (shot_number) float64 ...
    adc_end_timestamp       (shot_number) float64 ...
    adc_freqs               (shot_number) float64 ...
    B                       (probe, shot_number) float64 0.8 0.8 0.8 ... 0.8 0.8
    ts_temp                 (probe, shot_number) object 3.0018971964737067 ... nan
    ts_dens                 (probe, shot_number) object 1.5935223487373072e+19 ... nan
    fit_success_fl          (probe, shot_number) object True True ... True True
    temp                    (probe, shot_number) float64 3.183 3.152 ... 2.04
    d_temp                  (probe, shot_number) float64 0.03895 ... 0.08411
    isat                    (probe, shot_number) float64 0.07149 ... 0.00536
    d_isat                  (probe, shot_number) float64 0.0002479 ... 5.369e-05
    a                       (probe, shot_number) float64 0.02151 ... 0.08151
    d_a                     (probe, shot_number) float64 0.0004888 ... 0.0023
    v_f                     (probe, shot_number) float64 -24.46 ... -21.06
    d_v_f                   (probe, shot_number) float64 0.023 ... 0.04303
    dens                    (probe, shot_number) float64 6.528e+18 ... 3.98e+17
    d_dens                  (probe, shot_number) float64 4.378e+17 ... 2.298e+16
    chi2                    (probe, shot_number) float64 126.5 52.77 ... 91.35
    reduced_chi2            (probe, shot_number) float64 4.082 1.759 ... 2.855
    phi                     (prob

In [24]:
shifted_probes = magnum_probes.probe_position.copy()
for probe in magnum_probes.probe_position.keys():
    shifted_probes[probe] -= 1.75


### Thomson error recalculation

In [14]:
full_ts_ds = xr.open_dataset('full_ts_dataset.nc')
shot_number_ds = meta_data_ds.dropna('shot_number', subset=['adc_index']).swap_dims(
    {'shot_number':'adc_index'}).sel(adc_index=axial_scan_indices)
shot_numbers = shot_number_ds.shot_number

axial_ts = full_ts_ds.sel(ts_time=np.isin(full_ts_ds.shot_number, shot_numbers))

mean_percentage_dens_ts = (axial_ts['ts_d_density'] / axial_ts['ts_density']).mean('ts_time').values
mean_percentage_temp_ts = (axial_ts['ts_d_temperature'] / axial_ts['ts_temperature']).mean('ts_time').values

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [15]:
for shot_number in axial_ts.shot_number.values[0:1]:
    shot_ds = axial_ts.where(axial_ts.shot_number == shot_number, drop=True)
    md_shot_ds = meta_data_ds.sel(shot_number=shot_number)
    
    fig, ax = plt.subplots(2, sharex=True)
    fig.suptitle(f'{shot_number} - {md_shot_ds.shot_tilt.values}')
    
    ax[0].plot('ts_radial_pos', 'ts_d_temperature', data=md_shot_ds, label='averaged')
    ax[1].plot('ts_radial_pos', 'ts_d_density', data=md_shot_ds, label='averaged')
    
    ax[0].plot('ts_radial_pos', 'ts_temperature', data=shot_ds.std('ts_time') / np.sqrt(3), label='averaged_alt')
    ax[1].plot('ts_radial_pos', 'ts_density', data=shot_ds.std('ts_time') / np.sqrt(3), label='averaged_alt')
    
    for ts_time in shot_ds.ts_time.values:
        ds = shot_ds.sel(ts_time=ts_time)
        
        ax[0].plot('ts_radial_pos', 'ts_d_temperature', data=ds, label='raw')
        ax[1].plot('ts_radial_pos', 'ts_d_density', data=ds, label='raw')
        
        ax[0].plot(ds['ts_radial_pos'].values, ds['ts_temperature'].values * mean_percentage_temp_ts, label='estimated')
        ax[1].plot(ds['ts_radial_pos'].values, ds['ts_density'].values * mean_percentage_dens_ts, label='estimated')
        
    for axis in ax:
        axis.legend()
#         axis.set_yscale('log')

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [16]:
ts_ds = axial_ts.mean('ts_time')
ts_ds['ts_d_temperature'] = axial_ts.std('ts_time')['ts_temperature'] / np.sqrt(3)
ts_ds['ts_d_density'] = axial_ts.std('ts_time')['ts_density'] / np.sqrt(3)


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [17]:
fig, ax = plt.subplots(2, sharex=True)

ax[0].errorbar('ts_radial_pos', 'ts_temperature', yerr='ts_d_temperature', data=ts_ds)
ax[1].errorbar('ts_radial_pos', 'ts_density', yerr='ts_d_density', data=ts_ds)

<ErrorbarContainer object of 3 artists>

In [41]:
fig, ax = plt.subplots(1, 2, sharex=True, figsize=[10,5])
# ts_ds = ds_combined.sel(shot_number=127)
# ['ts_temperature'].plot.line(hue='shot_number', ax=ax[0])
# ds_combined.sel(shot_number=127)['ts_density'].plot.line(hue='shot_number', ax=ax[1])
ax[0].errorbar('ts_radial_pos', 'ts_temperature', yerr='ts_d_temperature', data=ts_ds, color='grey', 
               fmt='+', label='')
ax[1].errorbar('ts_radial_pos', 'ts_density', yerr='ts_d_density', data=ts_ds, color='grey', fmt='+', 
               label='')

# colours = {
#     'L': 'blue',
#     'S': 'red',
#     'B': 'green',
#     'R': 'gold'
# }
probe_colours = {
    'L': '#4477AA',
    'S': '#EE6677',
    'B': '#228833',
    'R': '#CCBB44'
}
for probe, pos in shifted_probes.items():
    probe = probe.upper()
#     pos -= 10
    ax[0].axvline(x=pos, linestyle='dashed', linewidth=0.75, color=probe_colours[probe], label=probe)
    ax[1].axvline(x=pos, linestyle='dashed', linewidth=0.75, color=probe_colours[probe], label=probe)
# ax[0].legend()
ax[1].legend(title='Probe Position')

# ax[0].set_ylabel(r'$T_e$ [eV]')
# ax[1].set_ylabel(r'$n_e$ [m$^{-3}$]')
ax[0].set_ylabel(r'Electron Temperature [eV]')
ax[1].set_ylabel(r'Electron Density [m$^{-3}$]')
ax[0].set_xlabel(r'TS Radial Position [mm]')
ax[1].set_xlabel(r'TS Radial Position [mm]')

fig.tight_layout()


In [21]:
# Temperature and Density measurements as a function of axis
fig, ax = plt.subplots(3)

for probe in ds_combined.probe.values:
    ds = ds_combined.sel(probe=probe).sortby('shot_target_pos')
    ax[0].errorbar('shot_target_pos', 'temp', yerr='d_temp', data=ds, label=probe, linestyle='none', marker='.')
    ax[1].errorbar('shot_target_pos', 'dens', yerr='d_dens', data=ds, label=probe, linestyle='none', marker='.')
    ax[2].plot('shot_target_pos', 'reduced_chi2', data=ds, label=probe)
    
ax[0].legend()
ax[1].legend()
ax[0].axhline(y=ds_combined['ts_temp'].values[0][0], **c.AX_LINE_DEFAULTS)
ax[1].axhline(y=ds_combined['ts_dens'].values[0][0], **c.AX_LINE_DEFAULTS)
ax[2].axhline(y=1.0, **c.AX_LINE_DEFAULTS)

In [22]:
ds_combined['shot_target_pos'] = -axial_scan_ds['shot_target_pos'] * 1000

In [42]:
fig, ax = plt.subplots(2, 3, sharex='col', sharey='row', figsize=[8.5,6])

# colours = {
#     'L': 'blue',
#     'S': 'red',
#     'B': 'green',
#     'R': 'gold'
# }
colours = probe_colours
labels = {
    'S': 'S',
    'L': 'L',
}

# ts_ds = ds_combined.sel(shot_number=127)
# ts_ds['ts_temperature'].plot.line(hue='shot_number', ax=ax[0][0], color='black', zorder=-1)
# ts_ds['ts_density'].plot.line(hue='shot_number', ax=ax[1][0], color='black', zorder=-1)
ax[0][0].errorbar('ts_radial_pos', 'ts_temperature', yerr='ts_d_temperature', data=ts_ds,
                  color='black', zorder=-1, label=None)
ax[1][0].errorbar('ts_radial_pos', 'ts_density', yerr='ts_d_density', data=ts_ds, 
                  color='black', zorder=-1, label=None)

for i, probe in enumerate(ds_combined.probe.values):
    ds = ds_combined.sel(probe=probe).sortby('shot_target_pos')
    ax[0][1+i].errorbar('shot_target_pos', 'temp', yerr='d_temp', data=ds, label=labels[probe], 
                        linestyle='none', marker='.', color=colours[probe])
    ax[1][1+i].errorbar('shot_target_pos', 'dens', yerr='d_dens', data=ds, label=labels[probe], 
                        linestyle='none', marker='.', color=colours[probe])

    ax[0][0].axvline(x=shifted_probes[probe.lower()], linestyle='dashed', linewidth=0.75, 
                     color=colours[probe], label=probe)
    ax[0][0].axvspan(
        shifted_probes[probe.lower()] + 2.5,
        shifted_probes[probe.lower()] - 2.5,
        color=colours[probe], alpha=0.4
    )
    ax[1][0].axvline(x=shifted_probes[probe.lower()], linestyle='dashed', linewidth=0.75, 
                     color=colours[probe], label=probe)
    ax[1][0].axvspan(
        shifted_probes[probe.lower()] + 2.5,
        shifted_probes[probe.lower()] - 2.5,
        color=colours[probe], alpha=0.4
    )
    
    ax[0][1+i].axhline(y=ts_ds.ts_temperature.interp(ts_radial_pos=shifted_probes[probe.lower()]).values, 
                     linestyle='dashed', linewidth=0.75, color=colours[probe], label='TS -'+labels[probe])
    ax[0][1+i].axhspan(
        ts_ds.ts_temperature.interp(ts_radial_pos=shifted_probes[probe.lower()]+2.5).values,
        ts_ds.ts_temperature.interp(ts_radial_pos=shifted_probes[probe.lower()]-2.5).values,
        color=colours[probe], alpha=0.4
    )
    
    ax[1][1+i].axhline(y=ts_ds.ts_density.interp(ts_radial_pos=shifted_probes[probe.lower()]).values , 
                     linestyle='dashed', linewidth=0.75, color=colours[probe], label='TS -'+labels[probe])
    ax[1][1+i].axhspan(
        ts_ds.ts_density.interp(ts_radial_pos=shifted_probes[probe.lower()]+2.5).values,
        ts_ds.ts_density.interp(ts_radial_pos=shifted_probes[probe.lower()]-2.5).values,
        color=colours[probe], alpha=0.4
    )

# ax[0][1].axhline(y=ts_ds.max('ts_radial_pos')['ts_temperature'].values, **c.AX_LINE_DEFAULTS, label='TS peak')
# ax[1][1].axhline(y=ts_ds.max('ts_radial_pos')['ts_density'].values, **c.AX_LINE_DEFAULTS, label='TS peak')
# ax[0][2].axhline(y=ts_ds.max('ts_radial_pos')['ts_temperature'].values, **c.AX_LINE_DEFAULTS, label='TS peak')
# ax[1][2].axhline(y=ts_ds.max('ts_radial_pos')['ts_density'].values, **c.AX_LINE_DEFAULTS, label='TS peak')
ax[1][0].legend()
ax[1][1].legend()
ax[0][0].legend()
ax[0][1].legend()
ax[0][2].legend()
ax[1][2].legend()

ax[0][0].set_title('(a)')
ax[0][1].set_title('(b)')
ax[0][2].set_title('(c)')

ax[1][0].set_title('')
ax[0][0].set_xlabel('')

ax[0][0].set_ylabel(r'Electron Temperature [eV]')
ax[1][0].set_ylabel(r'Electron Density [m$^{-3}$]')
ax[1][0].set_xlabel(r'TS Radial Position [mm]')
# ax[1][1].set_xlabel(r'Distance from TS Position [m]')
# ax[1][2].set_xlabel(r'Distance from TS Position [m]')
fig.text(0.69, 0.028, r'$z$ Position of Probes [mm]', ha='center')
fig.tight_layout()

In [21]:
ds_combined['d_temp_phi'] = np.abs(ds_combined['temp_phi'] - ds_combined['temp_fit_phi'])


In [28]:
# print(ds_combined['shot_target_pos'])
ds_combined.isel(shot_number=32)['shot_tilt']

<xarray.DataArray 'shot_tilt' ()>
array(9.986811)
Coordinates:
    shot_number  int32 175
    tilt         float64 10.0

In [24]:
fig, ax = plt.subplots(2, 3, sharex='col', sharey='row', constrained_layout=True)

colours = {
    'L': 'blue',
    'S': 'red',
    'B': 'green',
    'R': 'gold'
}

ts_ds = ds_combined.sel(shot_number=127)
ts_ds['ts_temperature'].plot.line(hue='shot_number', ax=ax[0][0], color='black', zorder=-1)
ts_ds['ts_density'].plot.line(hue='shot_number', ax=ax[1][0], color='black', zorder=-1)

for i, probe in enumerate(ds_combined.probe.values):
    ds = ds_combined.sel(probe=probe).sortby('shot_target_pos')
    ax[0][1+i].errorbar('shot_target_pos', 'temp', yerr='d_temp', data=ds, label=probe, linestyle='none', marker='.', color=colours[probe])
    ax[1][1+i].errorbar('shot_target_pos', 'dens', yerr='d_dens', data=ds, label=probe, linestyle='none', marker='.', color=colours[probe])
    
    ax[0][1+i].errorbar('shot_target_pos', 'temp_phi', yerr='d_temp_phi', data=ds, label=r'$T_e$ from $\phi$', linestyle='none', marker='^', color=colours[probe])
#     ax[0][1+i].plot('shot_target_pos', 'temp_fit_phi', data=ds, label=r'$T_e$ from $\phi$', linestyle='none', marker='^', color=colours[probe])

    ax[0][0].axvline(x=shifted_probes[probe.lower()], linestyle='dashed', linewidth=0.75, color=colours[probe], label=probe)
    ax[0][0].axvspan(
        shifted_probes[probe.lower()] + 2.5,
        shifted_probes[probe.lower()] - 2.5,
        color=colours[probe], alpha=0.4
    )
    ax[1][0].axvline(x=shifted_probes[probe.lower()], linestyle='dashed', linewidth=0.75, color=colours[probe], label=probe)
    ax[1][0].axvspan(
        shifted_probes[probe.lower()] + 2.5,
        shifted_probes[probe.lower()] - 2.5,
        color=colours[probe], alpha=0.4
    )
    
    ax[0][1+i].axhline(y=ts_ds.ts_temperature.interp(ts_radial_pos=shifted_probes[probe.lower()]).values, 
                     linestyle='dashed', linewidth=0.75, color=colours[probe], label=probe)
    ax[0][1+i].axhspan(
        ts_ds.ts_temperature.interp(ts_radial_pos=shifted_probes[probe.lower()]+2.5).values,
        ts_ds.ts_temperature.interp(ts_radial_pos=shifted_probes[probe.lower()]-2.5).values,
        color=colours[probe], alpha=0.4
    )
    
    ax[1][1+i].axhline(y=ts_ds.ts_density.interp(ts_radial_pos=shifted_probes[probe.lower()]).values , 
                     linestyle='dashed', linewidth=0.75, color=colours[probe], label='TS '+probe)
    ax[1][1+i].axhspan(
        ts_ds.ts_density.interp(ts_radial_pos=shifted_probes[probe.lower()]+2.5).values,
        ts_ds.ts_density.interp(ts_radial_pos=shifted_probes[probe.lower()]-2.5).values,
        color=colours[probe], alpha=0.4
    )

ax[0][1].axhline(y=ts_ds.max('ts_radial_pos')['ts_temperature'].values, **c.AX_LINE_DEFAULTS, label='TS peak')
ax[0][2].axhline(y=ts_ds.max('ts_radial_pos')['ts_temperature'].values, **c.AX_LINE_DEFAULTS, label='TS peak')
ax[1][1].axhline(y=ts_ds.max('ts_radial_pos')['ts_density'].values, **c.AX_LINE_DEFAULTS, label='TS peak')
ax[1][2].axhline(y=ts_ds.max('ts_radial_pos')['ts_density'].values, **c.AX_LINE_DEFAULTS, label='TS peak')
ax[1][0].legend()
ax[1][1].legend()
ax[0][0].legend()
ax[0][1].legend()

### Shifting the Probes
This section looks at the positions of the probes and shifts them, looking at how the ratio of TS measurements to Probe measurements changes and seeing if there is a minimum value of shift - hinting at the 'true' value of the probes relative to the Thomson coordinates which is likely not constant. 

In [52]:
shifts = np.linspace(-20, 20, 161)
print(shifts)

[-20.   -19.75 -19.5  -19.25 -19.   -18.75 -18.5  -18.25 -18.   -17.75
 -17.5  -17.25 -17.   -16.75 -16.5  -16.25 -16.   -15.75 -15.5  -15.25
 -15.   -14.75 -14.5  -14.25 -14.   -13.75 -13.5  -13.25 -13.   -12.75
 -12.5  -12.25 -12.   -11.75 -11.5  -11.25 -11.   -10.75 -10.5  -10.25
 -10.    -9.75  -9.5   -9.25  -9.    -8.75  -8.5   -8.25  -8.    -7.75
  -7.5   -7.25  -7.    -6.75  -6.5   -6.25  -6.    -5.75  -5.5   -5.25
  -5.    -4.75  -4.5   -4.25  -4.    -3.75  -3.5   -3.25  -3.    -2.75
  -2.5   -2.25  -2.    -1.75  -1.5   -1.25  -1.    -0.75  -0.5   -0.25
   0.     0.25   0.5    0.75   1.     1.25   1.5    1.75   2.     2.25
   2.5    2.75   3.     3.25   3.5    3.75   4.     4.25   4.5    4.75
   5.     5.25   5.5    5.75   6.     6.25   6.5    6.75   7.     7.25
   7.5    7.75   8.     8.25   8.5    8.75   9.     9.25   9.5    9.75
  10.    10.25  10.5   10.75  11.    11.25  11.5   11.75  12.    12.25
  12.5   12.75  13.    13.25  13.5   13.75  14.    14.25  14.5   14.75
  15. 

In [68]:
temp_ratios = {'s':[], 'l':[]}
dens_ratios = {'s':[], 'l':[]}

for shift in shifts:
    shifted_probes = magnum_probes.probe_position.copy()
    for probe in temp_ratios.keys():
        shifted_probes[probe] -= shift
        
        ds = ds_combined.sel(probe=probe.upper())
#         print(ds)
        
        interp_ts_temp = ts_ds.ts_temperature.interp(ts_radial_pos=shifted_probes[probe]).values
        interp_ts_dens = ts_ds.ts_density.interp(ts_radial_pos=shifted_probes[probe]).values
        
        temp_ratios[probe].append(ds.min('shot_number')['temp'] / interp_ts_temp)
        dens_ratios[probe].append(ds.max('shot_number')['dens'] / interp_ts_dens)
    

In [71]:
fig, ax = plt.subplots(3, sharex=True)

for probe in temp_ratios.keys():
    ax[0].plot(shifts, temp_ratios[probe], label=probe, color=colours[probe.upper()])
    ax[1].plot(shifts, dens_ratios[probe], label=probe, color=colours[probe.upper()])
    
ax[0].legend()
ax[1].legend()

# fig, ax = plt.subplots()

ax[2].plot(shifts, np.abs(np.array(temp_ratios['s']) - np.array(temp_ratios['l'])), label='Temp ratio difference')
ax[2].plot(shifts, np.array(dens_ratios['s']) - np.array(dens_ratios['l']), label='Dens ratio difference')
ax[2].set_yscale('log')
    
ax[2].legend()

## Measurement of Temperature from Knee

In [17]:
ds_combined['shot_number'].values

array([127, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
       155, 156, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
       182, 183, 184], dtype=int32)

In [33]:
fig, ax = plt.subplots()
# L,144
ds = ds_combined.sel(probe='S', shot_number=175)
ds.set_coords('voltage')['current'].plot.line(x='voltage', ax=ax)

In [34]:
iv_data = iv.IVData.from_dataset(ds)

In [36]:
sl_fitter = fts.StraightLineFitter()

In [37]:
fig, ax = plt.subplots()
iv_data.plot(ax=ax)
v_f = iv_data.get_vf()

ax.axvline(x=v_f, color='blue', zorder=0)
ax.axvline(x=-13.42, color='orangered', linestyle='dotted', linewidth=0.75, label='guess')
ax.legend()

ss_1_indices = slice(34,38)
ss_2_indices = slice(42,50)
extrap_1 = np.linspace(-23,-15,1000)
extrap_2 = np.linspace(-25,7,1000)
ax.plot(iv_data['V'][ss_1_indices], iv_data['I'][ss_1_indices], color='tab:orange', zorder=10)
ax.plot(iv_data['V'][ss_2_indices], iv_data['I'][ss_2_indices], color='tab:orange', zorder=10)

sl_fit_1 = sl_fitter.fit(iv_data['V'][ss_1_indices], iv_data['I'][ss_1_indices])
sl_fit_2 = sl_fitter.fit(iv_data['V'][ss_2_indices], iv_data['I'][ss_2_indices])
ax.plot(extrap_1, sl_fit_1.fit_function(extrap_1), color='red', linestyle='dashed')
ax.plot(extrap_2, sl_fit_2.fit_function(extrap_2), color='red', linestyle='dashed')

v_p = ((sl_fit_2.get_param('y_0') - sl_fit_1.get_param('y_0')) 
       / (sl_fit_1.get_param('m') - sl_fit_2.get_param('m')))
ax.axvline(x=v_p, color='red')


In [42]:
fig, ax = plt.subplots(2, sharex=True)
iv_data.plot(ax=ax[0])

v_f_index = iv_data.get_vf_index() - 4

grad_current = np.gradient(iv_data['I'])

# grad_2_current = np.gradient(grad_current)
# interp_grad_2 = interp1d(grad_2_current, iv_data['V'][v_f_index:])
v_p_alt = iv_data['V'][np.argmax(grad_current)]

ax[1].plot(iv_data['V'], grad_current)
# ax[1].plot(iv_data['V'][v_f_index:], grad_2_current)
# ax[1].plot(interp_grad_2(iv_data['I'][v_f_index:]), iv_data['I'][v_f_index:])
ax[1].axvline(x=v_p_alt, color='red')
# ax[1].axvline(x=interp_grad_2(0), color='orangered')
ax[1].axhline(y=0, **c.AX_LINE_DEFAULTS)

In [48]:
fig, ax = plt.subplots(3)

ax[0].plot(iv_data['t'], iv_data['V'], '-x')
# ax[0].twinx().plot(iv_data['t'], iv_data['I'], '-.')

current_spl = UnivariateSpline(iv_data['V'], iv_data['I'], s=1e-6, k=4)
dummy_x = np.linspace(iv_data['V'][0], iv_data['V'][-1], 10000)

iv_data.plot(ax=ax[0])
ax[0].plot(dummy_x, current_spl(dummy_x), zorder=10)

spline_grad = current_spl.derivative(n=1)
v_p_alt_2 = dummy_x[np.argmax(spline_grad(dummy_x))]

ax[1].plot(dummy_x, spline_grad(dummy_x))
ax[1].axvline(x=v_p_alt, color='orange', linewidth=0.5)
ax[1].axvline(x=v_p_alt_2, color='red', linewidth=0.5)
ax[1].set_ylabel(r'$I^{\prime}_P$')

spline_grad_2 = current_spl.derivative(n=2)
ax[2].plot(dummy_x, spline_grad_2(dummy_x))


ValueError: x must be strictly increasing

In [49]:
phi = iv_data.estimate_phi(plot_fl=True)
lp.estimate_temperature(v_f, phi)

4.418852528681764

In [52]:
def estimate_temperature(float_pot, plasma_pot, m_e=1.0, m_i=c.P_E_MASS_RATIO):
    return (float_pot - plasma_pot) / (np.log(0.6* np.sqrt(2 * np.pi * m_e / m_i)))
print(v_f, ds.v_f.values, v_p, v_p_alt)
print(estimate_temperature(v_f, v_p))
print(estimate_temperature(v_f, v_p_alt))
print(estimate_temperature(ds.v_f.values, v_p_alt))
# print(estimate_temperature(v_f, v_p_alt_2))

print(ds.temp.values, ts_ds.ts_temperature.interp(ts_radial_pos=shifted_probes[probe.lower()]).values)


-30.714376917311725 -31.35180194750439 -8.576429890301787 -15.912984276770347
6.609129666095241
4.418852528681764
4.609151325215067
3.43862347052968 0.9304985083116178


### Graph of Enjoyment of Hot Ones

In [53]:
gaussian = fts.GenericGaussianFitter()

a = np.linspace(0,24,50000)
b = (gaussian.fit_function(a, *[3, 0.75, 1.75]) 
     - gaussian.fit_function(a, *[0.5, 3.5, 12])
     - gaussian.fit_function(a, *[5, 0.2, 4])
     - gaussian.fit_function(a, *[4, 0.4, 4.4])
     - gaussian.fit_function(a, *[1, 0.1, 12])
     - gaussian.fit_function(a, *[0.5, 0.1, 16])
     - gaussian.fit_function(a, *[0.1, 0.1, 22])) + (0.04 * a)

with plt.xkcd():
    plt.figure()
    plt.plot(a, b, color='black')
    plt.xlabel('time (hours)')
    plt.ylabel('enjoyment of activity (arb)')